In [1]:
from randomgen import RandomNetworkGenerator
import json
import requests
import time

In [2]:
schema = json.load(open("../../../metadata/relations.json", "r"))
generator = RandomNetworkGenerator(schema)

custom_counts = {
    "BusinessUnit": 1,
    "ProductFamily": 1,
    "ProductOffering": 1,
    "Facility": 1,
    "Parts": 1,
    "Warehouse": 1,
    "Supplier": 1,
}

In [3]:
# API_URL = "https://bug-hardy-obviously.ngrok-free.app"
API_URL = "http://localhost:8000"

version = "static_small_10_v6"

In [ ]:
create_ops = generator.create_network(custom_counts)

print(f"Generated {len(create_ops)} operations")

print(create_ops)



In [ ]:
updated_create_ops = []

for op in create_ops:
    op["version"] = version 
    updated_create_ops.append(op)

# print(updated_create_ops)

# for op in updated_create_ops:
    # print(op)

# requests.post(f"{API_URL}/schema/live/update/bulk", json=updated_create_ops)

bulk_create_payload = {
    "version": version,
    "action": "bulk_create",
    "type": "schema",
    "timestamp": updated_create_ops[0]["timestamp"],
    "payload": []
}

for op in updated_create_ops:
    bulk_create_payload["payload"].append(op["payload"])

print(bulk_create_payload)

requests.post(f"{API_URL}/schema/live/update", json=bulk_create_payload)

In [ ]:
update_cycles = 10
for _ in range(update_cycles):
    update_ops = generator.generate_updates(
        node_updates=10,
        edge_updates=10
    )
    print(f"Generated {len(update_ops)} update operations")

    updated_update_ops = []

    for op in update_ops:
        op["version"] = version
        updated_update_ops.append(op)
    print(updated_update_ops)
    # requests.post(f"{API_URL}/schema/live/update/bulk", json=updated_update_ops)
    # time.sleep(1)

In [7]:
delete_cycles = 0
for _ in range(delete_cycles):
    delete_ops = generator.generate_deletions(
        node_deletions=5,
        edge_deletions=5
    )
    print(f"Generated {len(delete_ops)} delete operations")

    updated_delete_ops = []

    for op in delete_ops:
        op["version"] = version
        updated_delete_ops.append(op)

    requests.post(f"{API_URL}/schema/live/update/bulk", json=updated_delete_ops)